In [182]:
import tensorflow as tf
import tensorflow.keras as K
import tensorflow.keras.layers as L
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer 

In [183]:
class TrainingConfig():
    epochs = 5
    batch_size= 128
    learningRate=0.001
    valRate=0.1
    loss=['sparse_categorical_crossentropy']
    metrics=['accuracy']
    optimizer='adam'
    
class WordEmbedding():
    sequenceLength=256
    wordCount=10000
    wordDim=10

class DataSourceConfig():
    source=["./dataset/imdb_train_data.npy",
           "./dataset/imdb_test_data.npy",
           "./dataset/imdb_train_labels.npy",
           "./dataset/imdb_test_labels.npy"]

In [316]:
class ADLSTMModel(K.Model):
    def __init__(self,config,wordEmbedding):
        super(ADLSTMModel, self).__init__()
        self.model = None
        self.history = None
        self.config=config
        self.emb=wordEmbedding
        self.embedding=L.Embedding(input_dim=self.emb.wordCount,output_dim=self.emb.wordDim,input_length=self.emb.sequenceLength)
        self.lstm=L.LSTM(32,dropout=0.4)
        self.output_layer=L.Dense(2,activation="softmax")
        
    def get_adv_r(self,emb,y_true,rate=0.02):
        # 转化为常量意味着在tape中不用求导
        #emb=tf.convert_to_tensor(emb)
        with tf.GradientTape() as tp:
            #太难了，这是个tensor得watch才能记录
            tp.watch(emb)
            y=self.lstm(emb)
            y_pred=self.output_layer(y)
            loss=loss_fn_(y_true,y_pred)
        g=tp.gradient(y,emb)
        g=rate*tf.nn.l2_normalize(g,axis=2)
        #g=tf.convert_to_tensor(g)
        return g,loss
    
    def call(self, x,training=None,y_true=None):
        emb=self.embedding(x)
        #emb=tf.nn.l2_normalize(emb,axis=2)
        g=0
        if training is not None and y_true is not None:
            #g,loss=self.get_adv_r(emb,y_true)
            #self.add_loss(loss)
            pass
        
        ad_x=self.lstm(emb+g)
        output=self.output_layer(ad_x)
        return output

In [320]:
class AD_LSTM():
    def __init__(self,config,wordEmbedding):
        
        self.model = None
        self.history = None
        self.config=config
        self.emb=wordEmbedding
        self.name="AD_LSTM"
        
    def design_model(self,hiden_size=32,with_fc=False):
        
        self.model=ADLSTMModel(self.config,self.emb)

        #model.summary()
        return model
    
    def compile_model(self):
        #assert()
        if not self.model:
            print("Call design_modelXX() to build the model first.")
            return
        self.model.compile(optimizer=self.config.optimizer,loss=self.config.loss,metrics=self.config.metrics)

    def train_model(self,x,y):
        if not self.model:
            print("Call design_modelXX() to build the model first.")
            return
        self.compile_model()
        self.history=self.model.fit(x=x,y=y,batch_size=self.config.batch_size,epochs=self.config.epochs,validation_split=self.config.valRate)
        #self.model.compile()
        
    def train_model_custom(self,x,y):
        if not self.model:
            print("Call design_modelXX() to build the model first.")
        x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.1)
        print(x_train.shape)
        print(x_val.shape)
        train_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(self.config.batch_size)
        
        val_dataset = tf.data.Dataset.from_tensor_slices((x_val,y_val)).batch(self.config.batch_size)

        #loss
        loss_fn=K.losses.get(self.config.loss[0])
        #opt
        optimizer=K.optimizers.Adam()
        #metric
        train_acc_metric=K.metrics.SparseCategoricalAccuracy()#self.config.metrics[0])
        val_acc_metric  =K.metrics.SparseCategoricalAccuracy()#self.config.metrics[0])
        
        for times in range(self.config.epochs):
            step=0
            # 训练过程
            for x_batch_train, y_batch_train in train_dataset:
                with tf.GradientTape() as tp:
                    logits=self.model(x_batch_train)
                    loss=loss_fn(y_batch_train,logits)
                grads=tp.gradient(loss,self.model.trainable_weights)
                optimizer.apply_gradients(zip(grads, self.model.trainable_weights))
                train_acc_metric.update_state(y_batch_train,logits)
                
                if step % 50 == 0:
                    print('\r','Training loss (for one batch) at step %s: %s' % (step, float(tf.reduce_mean(loss))),end="",flush=True)
                    #print('Seen so far: %s samples' % ((step + 1) * self.config.batch_size))
                step+=1
            print()
            # 每个周期结束看一次
            train_acc = train_acc_metric.result()
            print('Training     acc over epoch %s: %s' % (times,float(train_acc)))
            # Reset training metrics at the end of each epoch
            train_acc_metric.reset_states()
            
            #验证过程
            for x_batch_val, y_batch_val in val_dataset:
                val_logits=self.model(x_batch_val)
                val_acc_metric.update_state(y_batch_val,val_logits)
            
            val_acc=val_acc_metric.result()
            print('Training val_acc over epoch %s: %s '%(times,float(val_acc)))
            val_acc_metric.reset_states()
    
    
    def save_model(self):
        if not self.model:
            print("Call design_modelXX() to build the model first.")
        pass
    
    def eval_model(self):
        if not self.model:
            print("Call design_modelXX() to build the model first.")
        pass
    
    def view_train(self):
        if not self.history:
            print("Model has not been trained, train it first")
            return
        
        train=self.history.history["loss"]
        valid=self.history.history["val_loss"]
        name='loss'
        plt.title('The %s with epoch runs'%name,fontsize=30)
        plt.xlabel('epoch',fontsize=20)
        plt.ylabel(name,fontsize=20)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        plt.plot(train,label=name)
        plt.plot(valid,label="val_"+name)
        plt.legend()
        plt.gcf().set_size_inches(15,4)
        plt.show()
    #model=K.models.Model(inputs=[input_layer],outputs=[output_layer])
    #model.compile('adam',loss='categorical_crossentropy',metrics=metrics)
    #return model

In [321]:
train_config=TrainingConfig()
word_embedding=WordEmbedding()
model=AD_LSTM(train_config,word_embedding)
model.design_model();

In [73]:
class Dataset():
    def __init__(self,config):
        self.config=config
    def load(self):
        L=[]
        for item in self.config.source:
            L.append(np.load(item,allow_pickle=True, fix_imports=True))
        return L

In [74]:
data=Dataset(DataSourceConfig)
train_data,test_data,train_labels,test_labels=data.load()

In [294]:
model.train_model_custom(train_data,train_labels)

In [ ]:
#model.train_model(x=train_data,y=train_labels)